# Download from Soundcloud using yt-dlp

## Setup the Notebook and the Dependencies

In [1]:
import yt_dlp
import os
from dotenv import load_dotenv
os.environ['PATH'] = r'C:\ffmpeg\bin;' + os.environ['PATH']

load_dotenv()
TOKEN = os.getenv('SC_TOKEN')
SONGLINK = "https://soundcloud.com/katyperry/last-friday-night-t-g-i-f"

## Download a track as .m4a using a token to access Soundcloud Go songs

In [ ]:
SONGLINK = "https://soundcloud.com/katyperry/last-friday-night-t-g-i-f"


ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',  # Output filename template
    'username': 'oauth',
    'password': TOKEN,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([SONGLINK])

## Get all the metadata to a song

In [ ]:
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(SONGLINK, download=False)
    
# Display all metadata
print("Available metadata:")
for key, value in info.items():
    print(f"{key}: {value}")


## Download as .mp3 with Thumbnail

Download the track, convert it into mp3. Get the hightest resolution thumbnail and embed it into the mp3 file. 

In [ ]:
SONGLINK = "https://soundcloud.com/skrillex/avicii-levels-skrillex-remix"

In [7]:
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(artist)s - %(title)s.%(ext)s',
    'username': 'oauth',
    'password': TOKEN,
    'postprocessors': [
        {
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        },
        {
            'key': 'FFmpegMetadata',
            'add_metadata': True,
        },
        {
            'key': 'EmbedThumbnail',
            'already_have_thumbnail': False,
        },
    ],
    'writethumbnail': True,  # Download thumbnail
    'embedthumbnail': True,  # Embed it in the audio file
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([SONGLINK])

[soundcloud] Extracting URL: https://soundcloud.com/michael-big-daddy-day/captain-jack
[soundcloud] michael-big-daddy-day/captain-jack: Downloading info JSON
[soundcloud] 229665828: Downloading hls_aac format info JSON
[soundcloud] 229665828: Downloading hls_aac format info JSON
[soundcloud] 229665828: Downloading http_aac format info JSON
[soundcloud] 229665828: Downloading hls_aac format info JSON
[soundcloud] 229665828: Downloading hls_mp3 format info JSON
[soundcloud] 229665828: Downloading http_mp3 format info JSON
[info] 229665828: Downloading 1 format(s): http_aac_hq
[info] Downloading video thumbnail original ...
[info] Writing video thumbnail original to: NA - Captain Jack.jpg
[download] Destination: NA - Captain Jack.m4a
[download] 100% of    6.12MiB in 00:00:05 at 1.04MiB/s   
[FixupM4a] Correcting container of "NA - Captain Jack.m4a"
[ExtractAudio] Destination: NA - Captain Jack.mp3
Deleting original file NA - Captain Jack.m4a (pass -k to keep)
[Metadata] Adding metadata to

# Add the songlink as a comment
When downloading a song, also embed the songlink into the mp3 as the comment tag

In [ ]:
SONGLINK = "https://soundcloud.com/priincemuzik/moliy-silent-addy-shake-it-to-the-max-remix-feat-skillibeng-shenseea"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'username': 'oauth',
    'password': TOKEN,
    'postprocessors': [
        {
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        },
        {
            'key': 'FFmpegMetadata',
            'add_metadata': True,
        },
        {
            'key': 'EmbedThumbnail',
        },
    ],
    'writethumbnail': True,
    'embedthumbnail': True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(SONGLINK, download=True)
    
    artist = info.get('artist', '').strip()
    title = info['title']
    
    # Use only title if no artist is available
    if artist and artist != '':
        mp3_file = f"{artist} - {title}.mp3"
    else:
        mp3_file = f"{title}.mp3"

In [ ]:
from mutagen.id3 import ID3, COMM
# Add comment with the song URL to the downloaded file
audio = ID3(mp3_file)
audio.add(COMM(text=[SONGLINK]))
audio.save()

# Download a Playlist

In [ ]:
from mutagen.id3 import ID3, COMM
PLAYLIST_LINK = "https://soundcloud.com/user-251038582/sets/tims-techno-remix-picks"
START_FROM_TRACK = 69  # Change to desired starting track number (1-based index)

ydl_opts_playlist = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'username': 'oauth',
    'password': TOKEN,

    'playlist_start': START_FROM_TRACK,

    'postprocessors': [
        {
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        },
        {
            'key': 'FFmpegMetadata',
            'add_metadata': True,
        },
        {
            'key': 'EmbedThumbnail',
        },
    ],
    'writethumbnail': True,
    'embedthumbnail': True,
}

with yt_dlp.YoutubeDL(ydl_opts_playlist) as ydl:
    playlist_info = ydl.extract_info(PLAYLIST_LINK, download=True)
    
    # Add song URL as comment to each downloaded track
    for entry in playlist_info['entries']:
        artist = entry.get('artist', '').strip()
        title = entry['title']
        
        # Use only title if no artist is available
        if artist and artist != '':
            mp3_file = f"{artist} - {title}.mp3"
        else:
            mp3_file = f"{title}.mp3"
        
        song_url = entry['webpage_url']
        
        audio = ID3(mp3_file)
        audio.delall('TALB')  # Remove album tag, would otherwise be set to playlist title
        audio.add(COMM(text=[song_url]))
        audio.save()
        
print(f"Downloaded {len(playlist_info['entries'])} tracks from playlist")

# Check for native download

In [ ]:
SONGLINK = "https://soundcloud.com/fawafawa/liverpool-street-in-the-rain-speed-garage-edit"

ydl_opts_native = {
    'format': 'original/best',  # Prioritize original/native download over streams
    'outtmpl': '%(artist)s - %(title)s.%(ext)s',
    'username': 'oauth',
    'password': TOKEN,
}

with yt_dlp.YoutubeDL(ydl_opts_native) as ydl:
    info = ydl.extract_info(SONGLINK, download=False)
    
    # Check if original download is available
    if info.get('download_url'):
        print(f"Native download available: {info.get('ext')} format")
    else:
        print("No native download available, will use stream")
    
    ydl.download([SONGLINK])

# Buy / External Download available?



In [ ]:
SONGLINK = "https://soundcloud.com/taziaus/way-2-sexy-tazi-bootleg"

# Check using yt-dlp (will fail to detect)
with yt_dlp.YoutubeDL({'username': 'oauth', 'password': TOKEN}) as ydl:
    info = ydl.extract_info(SONGLINK, download=False)
    
    # Check if external purchase/download link is available
    has_external_link_ytdlp = bool(info.get('purchase_url'))
    
    print("=== Using yt-dlp ===")
    print(f"External buy/download link available: {has_external_link_ytdlp}")
    if has_external_link_ytdlp:
        print(f"Link: {info.get('purchase_url')}")
    else:
        print("yt-dlp does NOT extract purchase_url for SoundCloud")

In [ ]:
# Verify all available metadata fields
print("All available metadata fields:")
for key in sorted(info.keys()):
    print(f"  {key}")

# Check specifically for purchase/download related fields
purchase_fields = [k for k in info.keys() if 'purchase' in k.lower() or 'buy' in k.lower() or 'external' in k.lower()]
print(f"\nPurchase/Buy/External related fields: {purchase_fields if purchase_fields else 'None found'}")
print(f"\npurchase_url value: {info.get('purchase_url')}")

## Web Scraping Solution for External Download Links

**Note:** yt-dlp does NOT extract `purchase_url` for SoundCloud tracks, even when they have "Free Download" buttons. These external links (to Hypeddit, Toneden, etc.) are embedded in the webpage HTML but not exposed through SoundCloud's API.

We need to scrape the SoundCloud page HTML directly to find the `purchaseLink_container` div.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def check_soundcloud_external_download(track_url):
    """
    Check if a SoundCloud track has an external download link (Free Download button).
    
    Args:
        track_url: SoundCloud track URL
        
    Returns:
        dict with 'has_external_link' (bool) and 'external_link' (str or None)
    """
    try:
        # Fetch the SoundCloud page HTML
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        response = requests.get(track_url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for the embedded JSON data in script tags
        # SoundCloud embeds track data in window.__sc_hydration
        for script in soup.find_all('script'):
            if script.string and 'window.__sc_hydration' in script.string:
                # Search for purchase_url in the JSON data
                match = re.search(r'["\']purchase_url["\']\s*:\s*["\']([^"\']+)', script.string)
                if match:
                    external_url = match.group(1)
                    # Decode unicode escapes if present
                    external_url = external_url.encode().decode('unicode_escape')
                    return {
                        'has_external_link': True,
                        'external_link': external_url
                    }
        
        return {'has_external_link': False, 'external_link': None}
        
    except Exception as e:
        print(f"Error checking external link: {e}")
        return {'has_external_link': False, 'external_link': None, 'error': str(e)}

In [ ]:
# Test with the track that has a Free Download button
test_url = "https://soundcloud.com/taziaus/way-2-sexy-tazi-bootleg"

result = check_soundcloud_external_download(test_url)

print(f"Has external download link: {result['has_external_link']}")
if result['has_external_link']:
    print(f"External link: {result['external_link']}")
else:
    print("No external download link found")
    if 'error' in result:
        print(f"Error: {result['error']}")

In [ ]:
# Debug: Let's examine the HTML to understand the structure
test_url = "https://soundcloud.com/taziaus/way-2-sexy-tazi-bootleg"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
response = requests.get(test_url, headers=headers, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')

# Search for any element containing "purchase" or "download"
print("=== Searching for purchase/download related elements ===\n")

# Look for any div with purchase in class name
purchase_divs = soup.find_all('div', class_=lambda x: x and 'purchase' in x.lower())
print(f"Divs with 'purchase' in class: {len(purchase_divs)}")
for div in purchase_divs[:3]:
    print(f"  Class: {div.get('class')}")
    print(f"  Content preview: {str(div)[:200]}...\n")

# Look for links with "download" text
download_links = soup.find_all('a', string=lambda x: x and 'download' in x.lower())
print(f"\nLinks with 'download' in text: {len(download_links)}")
for link in download_links[:3]:
    print(f"  Text: {link.get_text(strip=True)}")
    print(f"  Href: {link.get('href')}")
    print(f"  Classes: {link.get('class')}\n")

In [ ]:
# Look for JSON data embedded in script tags
import json
import re

print("=== Searching for embedded JSON data ===\n")

# Find all script tags
scripts = soup.find_all('script')
print(f"Total script tags found: {len(scripts)}\n")

# Look for hydration data or initial state
for i, script in enumerate(scripts):
    script_content = script.string
    if script_content and ('purchase' in script_content.lower() or 'buy_link' in script_content.lower() or 'external_url' in script_content.lower()):
        print(f"Script {i} contains purchase/buy/external_url keywords")
        print(f"Preview: {script_content[:500]}...\n")
        
# Try to find the main data payload
for script in scripts:
    if script.string and 'window.__sc_hydration' in script.string:
        print("Found window.__sc_hydration!")
        # Try to extract and search for purchase_url
        match = re.search(r'purchase_url["\']?\s*:\s*["\']([^"\']+)', script.string)
        if match:
            print(f"Found purchase_url: {match.group(1)}")
        else:
            print("No purchase_url found in hydration data")
        break

In [ ]:
# Test with multiple tracks to verify the function
test_tracks = [
    ("https://soundcloud.com/taziaus/way-2-sexy-tazi-bootleg", "Track with Free Download"),
    ("https://soundcloud.com/katyperry/last-friday-night-t-g-i-f", "Track without Free Download"),
]

print("=== Testing External Download Detection ===\n")
for track_url, description in test_tracks:
    print(f"{description}:")
    print(f"  URL: {track_url}")
    
    result = check_soundcloud_external_download(track_url)
    
    if result['has_external_link']:
        print(f"  ✓ External download link found: {result['external_link']}")
    else:
        print(f"  ✗ No external download link")
    print()

## Summary: yt-dlp vs Web Scraping

**yt-dlp limitations:**
- `purchase_url` field is **NOT** extracted by yt-dlp's SoundCloud extractor
- Returns `None` even when tracks have "Free Download" buttons
- yt-dlp only uses SoundCloud's API, which doesn't expose external download links

**Web scraping solution:**
- Successfully extracts `purchase_url` from the embedded JSON in `window.__sc_hydration`
- Works for tracks with external download links (Hypeddit, Toneden, etc.)
- Returns `None` for tracks without external links (as expected)

**Recommendation:** Use the `check_soundcloud_external_download()` function above when you need to detect external download links on SoundCloud tracks.

# Download a Playlist but wait in between

In [ ]:
from mutagen.id3 import ID3, COMM
PLAYLIST_LINK = "https://soundcloud.com/user-251038582/sets/tims-minimal-house"

ydl_opts_playlist = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'username': 'oauth',
    'password': TOKEN,
    'postprocessors': [
        {
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        },
        {
            'key': 'FFmpegMetadata',
            'add_metadata': True,
        },
        {
            'key': 'EmbedThumbnail',
        },
    ],
    'writethumbnail': True,
    'embedthumbnail': True,
    
    # Rate limiting configuration to stay under 600 requests per 10 minutes
    # Allows ~50 tracks per 10 minutes, ~300 tracks per hour
    'sleep_requests': 1,           # 1 second between API requests during metadata extraction
    'sleep_interval': 5,             # Minimum 5 seconds between track downloads
    'max_sleep_interval': 20,         # Maximum 20 seconds between track downloads
    'extractor_retries': 10,         # Allow up to 10 retries for rate limit errors
    'retry_sleep': 'extractor:exp=1:120',  # Exponential backoff (1-120 seconds) on rate limit
    
    # Error handling - continue downloading even if individual tracks fail
    'ignoreerrors': True,            # Skip unavailable/failed tracks and continue with the rest
    'no_warnings': False,            # Still show warnings about failed tracks
}

with yt_dlp.YoutubeDL(ydl_opts_playlist) as ydl:
    playlist_info = ydl.extract_info(PLAYLIST_LINK, download=True)
    
    # Add song URL as comment to each downloaded track
    for entry in playlist_info['entries']:
        if entry is None:  # Skip entries that failed to download
            continue
            
        artist = entry.get('artist', '').strip()
        title = entry['title']
        
        mp3_file = f"{title}.mp3"
        
        song_url = entry['webpage_url']
        
        try:
            audio = ID3(mp3_file)
            audio.delall('TALB')  # Remove album tag, would otherwise be set to playlist title
            audio.add(COMM(text=[song_url]))
            audio.save()
        except Exception as e:
            print(f"Warning: Could not add metadata to {mp3_file}: {e}")
            continue
        
print(f"Downloaded {len([e for e in playlist_info['entries'] if e is not None])} tracks from playlist")

In [ ]:
import csv
import random
import time
from pathlib import Path

def analyze_playlist_download_options(
    playlist_url: str,
    token: str,
    output_dir: str = "downloads/playlist",
    log_csv: str = "playlist_download_log.csv",
) -> None:
    """
    Analyze a SoundCloud playlist, log download options per track, and download via yt-dlp when needed.

    Rules:
    - Prefer native SoundCloud download when available (do NOT download via yt-dlp).
    - If native download is not available, download via yt-dlp (even if external link exists).
    - Always log availability and yt-dlp success/failure.
    """
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    log_path = Path(log_csv)
    is_new_log = not log_path.exists()
    
    def sleep_between_tracks():
        time.sleep(random.uniform(5, 20))
    
    # Step 1: get playlist entries with minimal requests (flat extraction)
    ydl_opts_list = {
        "extract_flat": True,
        "skip_download": True,
        "username": "oauth",
        "password": token,
        "sleep_interval_requests": 5,
        "max_sleep_interval_requests": 20,
        "extractor_retries": 10,
        "retry_sleep": "extractor:exp=1:120",
        "ignoreerrors": True,
        "no_warnings": False,
    }
    
    # Step 2: per-track info (rate-limited requests)
    ydl_opts_info = {
        "format": "original/best",
        "username": "oauth",
        "password": token,
        "sleep_interval_requests": 5,
        "max_sleep_interval_requests": 20,
        "extractor_retries": 10,
        "retry_sleep": "extractor:exp=1:120",
        "ignoreerrors": True,
        "no_warnings": False,
    }
    
    # Step 3: download with SAME rate-limited options as the playlist download cell (Cell 28)
    ydl_opts_download = {
        "format": "bestaudio/best",
        "outtmpl": str(output_path / "%(title)s.%(ext)s"),
        "username": "oauth",
        "password": token,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "320",
            },
            {
                "key": "FFmpegMetadata",
                "add_metadata": True,
            },
            {
                "key": "EmbedThumbnail",
            },
        ],
        "writethumbnail": True,
        "embedthumbnail": True,
        "sleep_requests": 5,
        "sleep_interval": 5,
        "max_sleep_interval": 20,
        "sleep_interval_requests": 5,
        "max_sleep_interval_requests": 20,
        "extractor_retries": 10,
        "retry_sleep": "extractor:exp=1:120",
        "ignoreerrors": True,
        "no_warnings": False,
    }
    
    with open(log_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "track_title",
                "track_url",
                "native_download_available",
                "external_link_available",
                "external_link",
                "ytdlp_downloaded",
                "ytdlp_error",
            ],
        )
        if is_new_log:
            writer.writeheader()
        
        # Get playlist entries flat to avoid bulk metadata requests
        with yt_dlp.YoutubeDL(ydl_opts_list) as ydl_list:
            playlist_info = ydl_list.extract_info(playlist_url, download=False)
            entries = playlist_info.get("entries", []) if playlist_info else []
        
        with yt_dlp.YoutubeDL(ydl_opts_info) as ydl:
            for entry in entries:
                if not entry:
                    continue
                track_url = entry.get("url") or entry.get("webpage_url")
                if not track_url:
                    continue
                
                # Fetch full track info (rate-limited per request)
                try:
                    info = ydl.extract_info(track_url, download=False)
                except Exception as e:
                    writer.writerow({
                        "track_title": entry.get("title", ""),
                        "track_url": track_url,
                        "native_download_available": False,
                        "external_link_available": False,
                        "external_link": "",
                        "ytdlp_downloaded": False,
                        "ytdlp_error": f"info_error: {e}",
                    })
                    sleep_between_tracks()
                    continue
                
                if not info:
                    writer.writerow({
                        "track_title": entry.get("title", ""),
                        "track_url": track_url,
                        "native_download_available": False,
                        "external_link_available": False,
                        "external_link": "",
                        "ytdlp_downloaded": False,
                        "ytdlp_error": "info_error: unavailable (None)",
                    })
                    sleep_between_tracks()
                    continue
                
                title = info.get("title", "")
                native_available = bool(info.get("download_url"))
                
                external_link = ""
                external_available = False
                try:
                    ext_result = check_soundcloud_external_download(track_url)
                    external_available = bool(ext_result.get("has_external_link"))
                    external_link = ext_result.get("external_link") or ""
                except Exception:
                    pass
                
                # If native download is available, prefer it (no yt-dlp download)
                if native_available:
                    writer.writerow({
                        "track_title": title,
                        "track_url": track_url,
                        "native_download_available": True,
                        "external_link_available": external_available,
                        "external_link": external_link,
                        "ytdlp_downloaded": False,
                        "ytdlp_error": "",
                    })
                    sleep_between_tracks()
                    continue
                
                # Otherwise download with yt-dlp (even if external link exists)
                ytdlp_ok = False
                ytdlp_err = ""
                try:
                    with yt_dlp.YoutubeDL(ydl_opts_download) as ydl_dl:
                        ydl_dl.download([track_url])
                    ytdlp_ok = True
                except Exception as e:
                    ytdlp_err = str(e)
                
                writer.writerow({
                    "track_title": title,
                    "track_url": track_url,
                    "native_download_available": False,
                    "external_link_available": external_available,
                    "external_link": external_link,
                    "ytdlp_downloaded": ytdlp_ok,
                    "ytdlp_error": ytdlp_err,
                })
                sleep_between_tracks()

In [ ]:
# Example usage
PLAYLIST_LINK = "https://soundcloud.com/user-251038582/sets/10yeardancerobix"
analyze_playlist_download_options(PLAYLIST_LINK, TOKEN)

[soundcloud:set] Extracting URL: https://soundcloud.com/user-251038582/sets/10yeardancerobix
[soundcloud:set] user-251038582/sets/10yeardancerobix: Downloading JSON metadata
[download] Downloading playlist: #10YEARdancerobix
[soundcloud:set] Playlist #10YEARdancerobix: Downloading 243 items of 243
[download] Downloading item 1 of 243
[download] Downloading item 2 of 243
[download] Downloading item 3 of 243
[download] Downloading item 4 of 243
[download] Downloading item 5 of 243
[download] Downloading item 6 of 243
[download] Downloading item 7 of 243
[download] Downloading item 8 of 243
[download] Downloading item 9 of 243
[download] Downloading item 10 of 243
[download] Downloading item 11 of 243
[download] Downloading item 12 of 243
[download] Downloading item 13 of 243
[download] Downloading item 14 of 243
[download] Downloading item 15 of 243
[download] Downloading item 16 of 243
[download] Downloading item 17 of 243
[download] Downloading item 18 of 243
[download] Downloading ite

ERROR: [WinError 5] Zugriff verweigert: 'downloads\\playlist\\Icona Pop： I Love It (feat. Charli XCX).temp.mp3' -> 'downloads\\playlist\\Icona Pop： I Love It (feat. Charli XCX).mp3'


[soundcloud] Extracting URL: https://soundcloud.com/pitbull/pitbull-ne-yo-time-of-our
[soundcloud] Sleeping 5 seconds ...
[soundcloud] pitbull/pitbull-ne-yo-time-of-our: Downloading info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading hls_aac format info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading http_aac format info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading hls_mp3 format info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading http_mp3 format info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading hls_opus format info JSON
[soundcloud] Extracting URL: https://soundcloud.com/pitbull/pitbull-ne-yo-time-of-our
[soundcloud] pitbull/pitbull-ne-yo-time-of-our: Downloading info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: Downloading hls_aac format info JSON
[soundcloud] Sleeping 5 seconds ...
[soundcloud] 1640518122: 